In [ ]:
import pandas as pd
from sqlalchemy import create_engine, Table, Column, Integer, String, Date, MetaData

engine = create_engine('sqlite:///hospital_data.db', echo=True)
metadata = MetaData()

def create_staging_table():
    staging_table = Table('Staging_Customers', metadata,
        Column('Customer_Name', String(255)),
        Column('Customer_Id', String(18), primary_key=True),
        Column('Open_Date', Date),
        Column('Last_Consulted_Date', Date),
        Column('Vaccination_Id', String(5)),
        Column('Dr_Name', String(255)),
        Column('State', String(5)),
        Column('Country', String(5)),
        Column('DOB', Date),
        Column('Is_Active', String(1)),
    )
    metadata.create_all(engine)
    print("Staging Table Created")

def create_country_table(country_name):
    country_table = Table(f"Table_{country_name}", metadata,
        Column('Customer_Name', String(255)),
        Column('Customer_Id', String(18), primary_key=True),
        Column('Open_Date', Date),
        Column('Last_Consulted_Date', Date),
        Column('Vaccination_Id', String(5)),
        Column('Dr_Name', String(255)),
        Column('State', String(5)),
        Column('Country', String(5)),
        Column('DOB', Date),
        Column('Is_Active', String(1)),
        Column('Age', Integer),
        Column('Days_Since_Last_Consulted', Integer),
    )
    metadata.create_all(engine)
    print(f"Table_{country_name} Created")

def insert_into_country_table(country_name, df_country):
    table = Table(f"Table_{country_name}", metadata, autoload_with=engine)
    with engine.connect() as connection:
        for index, row in df_country.iterrows():
            insert_query = table.insert().values(
                Customer_Name=row['Customer_Name'],
                Customer_Id=row['Customer_Id'],
                Open_Date=row['Open_Date'],
                Last_Consulted_Date=row['Last_Consulted_Date'],
                Vaccination_Id=row['Vaccination_Id'],
                Dr_Name=row['Dr_Name'],
                State=row['State'],
                Country=row['Country'],
                DOB=row['DOB'],
                Is_Active=row['Is_Active'],
                Age=row['Age'],
                Days_Since_Last_Consulted=row['Days_Since_Last_Consulted']
            )
            connection.execute(insert_query)
    print(f"Data Inserted into Table_{country_name}")

def read_and_process_csv(file_path, country_name):
    df = pd.read_csv(file_path)
    df["DOB"] = pd.to_datetime(df["DOB"])
    df["Last_Consulted_Date"] = pd.to_datetime(df["Last_Consulted_Date"])
    df["Open_Date"] = pd.to_datetime(df["Open_Date"])

    current_date = pd.Timestamp.now()
    df["Age"] = current_date.year - df["DOB"].dt.year
    df["Days_Since_Last_Consulted"] = (current_date - df["Last_Consulted_Date"]).dt.days
    df = df[df["Days_Since_Last_Consulted"] > 30]

    insert_into_country_table(country_name, df)

create_staging_table()
create_country_table('USA')
create_country_table('IND')
create_country_table('AUS')

read_and_process_csv('USA.csv', 'USA')
read_and_process_csv('IND.csv', 'IND')
read_and_process_csv('AUS.csv', 'AUS')
